# Problem 1
## a. Using variable elimination, calculate the probability that a student who did well on the exam understands the material.P(+u|+e). Show your work.

First grabbing i variable

P(i)

| I  | Probability |
| -- | ----------- |
| -i | 0.3         |
| +i | 0.7         |

Next Joining With T

P(i,t)

| I | T | Probability     |
|---|---|-----------------|
|+i | +t| 0.8*0.7 = 0.56  |
| -i| +t| 0.5*0.3 = 0.16  |

sum = 0.56+0.16 = 0.72

Eliminiating I


P(t)

| T | Probability     |
|---|-----------------|
| +t| sum = 0.72      |
| +t| 1-sum = 0.28    |



Inspecting h

P(h)

| h  | Probability |
| -- | ----------- |
| -h | 0.4         |
| +h | 0.6         |

Joining with h and i

P(u,i,h)

| u | i | h | Probability         |
|---|---|---|---------------------|
|+u | +i| +h| 0.9*0.6*0.7 = 0.378 |
|+u | +i| -h| 0.3*0.7*0.4 = 0.084 |
|+u | -i| +h| 0.7*0.3*0.6 = 0.126 |
|+u | -i| -h| 0.3*0.3*0.4 = 0.036 |

 sum = 0.378+0.084+0.126+0.036 = 0.624


| u  | Probability |
| -- | ----------- |
| +u | sum = 0.624        |
| -u | 1 - sum = 0.376         |

Joining e with u and t

P(e,t,u)

| e | t | u | Probability             |
|---|---|---|------------------------ |
|+e | +t| +u| 0.9*0.72*0.624 = 0.4043 |
|+e | +t| -u| 0.5*0.72*0.376 = 0.1354 |
|+e | -t| +u| 0.7*0.28*0.624 = 0.1223 |
|+e | -t| -u| 0.3*0.28*0.376 = 0.0316 |

Eliminating t

| e | u | Probability             |
|---|---|------------------------ |
|+e | +u| P(+e,+t,+u)+P(+e,-t,+u) = (0.4043 + 0.1223) = 0.5266 |
|+e | -u| P(+e,+t,-u)+P(+e,-t,-u) = (0.1354+0.0316) = 0.167 |

Normalizing e

| e | Probability             |
|---|------------------------ |
|+e |+u | P(+e,+u)/P(+u) = 0.5266/0.624 = 0.844 |





## b. Given the Bayesian network, are T and U independent? Why
T and U are not independent because they both depend on i.
## c. Are I and H conditionally independent given E? Why
I and H are not conditionally independent because I depends on H.
## d. Are E and H conditionally independent given U? Why

E and H are not conditionally independent given U because H is a parent of U and E is a parent of U.

## e. Are T and H independent? Why
    T and H are independent because they do not share a parent.

# Problem 2

In [10]:
import numpy as np
import pandas as pd
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from IPython.display import display, Markdown
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import cross_val_score
import statistics
data = pd.read_csv("Life Expectancy Data.csv")
data.describe()


,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


## a. Divide the data into a training set and a testing set
I am dividing the training set and testing set by selecting 25% of the data for training (with replacement) and 75% as the test data set.


In [11]:
train = data.sample(frac=0.80)
test = data.sample(frac=0.20)

## b. Using your training set, determine which variables are actually affecting the life expectancy.How did you arrive at that conclusion?

In [12]:
# To find correlation of each variable perform linear regression in relation to each variable
def print_col_correlations(data):
    filtered = data.values[:,:]
    filtered=filtered[:,4:]
    i = 4
    r_squared_data = []
    for col in filtered.T:
        col = col.astype(float)
        col = np.array(col)

        y =  data.values[:,3]

        y = y.astype(float)
        nans = np.isnan(y)
        y = y[~nans]

        col = col[~nans]

        col_nans = np.isnan(col)
        col=col[~col_nans]
        y = y[~col_nans]
        reg = LinearRegression().fit(col.reshape(-1,1), y)
        #print("score for: "+str(data.columns[i]))
        score = reg.score(col.reshape(-1,1), y)
        r_squared_data.append([data.columns[i],score])
        i+=1
    return pd.DataFrame(
        r_squared_data,columns=["measure_name","r squared"])
print_col_correlations(train)


,measure_name,r squared
0,Adult Mortality,0.470402
1,infant deaths,0.042218
2,Alcohol,0.168730
3,percentage expenditure,0.138077
4,Hepatitis B,0.065633
5,Measles,0.030214
6,BMI,0.312562
7,under-five deaths,0.053714
8,Polio,0.214009
9,Total expenditure,0.047994


Infant mortality ,Hepatitis B, Measles, under-five deaths, Total expenditure and Population	have low correlation with life expectancy. Therefore they will be removed from the analysis. I arrived at that conclusion by finding the linear correlation between the variable and life expectancy. The r^2 values are shown above. The measures with low r^2 values were dropped. 

In [13]:
def drop_rows(data):
    data = data.dropna()
    data.drop('infant deaths',1)
    data.drop('Hepatitis B',1)
    data.drop('Measles ',1)
    data.drop('under-five deaths ',1)
    data.drop('Total expenditure',1)
    data.drop('Population',1)
    return data
def make_model(data,normalize=False):
    
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = LinearRegression(normalize=normalize).fit(data.values[:,4:], life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg

make_model(train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Evaluate how well does your model predict life expectancy


In [14]:
def eval_model(model,train):

    # dropping rows from test dataset
    test_cleaned = drop_rows(test)
    x = test_cleaned.values[:,4:]
    y=test_cleaned.values[:,3]
    #for i in model.coef_:
    #    print(i)
    #for i in test_cleaned.axes[0]:
    #    print(i)
    data = []
    try:
        for coef,axes in zip(model.coef_,test_cleaned.axes[1]):
            data.append([axes,coef])
        frame = pd.DataFrame(data,columns=["Axis Name","Coefficient"])
        display(frame)
    except:
        a=0

    countries = {}
    for row in test_cleaned.values:
        #print(row[0])
        #print(row[4:])
        prediction = model.predict(row[4:].reshape(1, -1))[0]
        actual = row[3]
        err = prediction-actual
        #print(repr(err))
        try:
            countries[row[0]].append(err)
        except:
            countries[row[0]]=[err]
    #print(countries)
    country_stdev_arr = []
    for country in countries:
        stdev=0
        if(len(countries[country])>1):
            stdev = statistics.stdev(countries[country])
        #print(stdev)
        country_stdev_arr.append([country,stdev])
    display(pd.DataFrame(country_stdev_arr,columns=["Country","Standard Deviation"]))
model = make_model(train)
eval_model(model,train)

,Axis Name,Coefficient
0,Country,-1.661117e-02
1,Year,9.021516e-02
2,Status,-2.581156e-02
3,Life expectancy,4.442752e-04
4,Adult Mortality,-3.231234e-03
5,infant deaths,-5.989727e-06
6,Alcohol,2.932094e-02
7,percentage expenditure,-6.868246e-02
8,Hepatitis B,8.696151e-03
9,Measles,6.220529e-02


,Country,Standard Deviation
0,Greece,0.713439
1,Morocco,2.634429
2,Jamaica,0.873763
3,Seychelles,3.115851
4,France,2.167942
...,...,...
118,Costa Rica,0.000000
119,Syrian Arab Republic,0.000000
120,Bosnia and Herzegovina,6.101516
121,Albania,0.000000


The results show the standard deviation for the difference between real life expectancy and real life expectancy based on country. The error is small for most countries, this means that the model is fairly good at perdicting life expctancy. 

# Scikit-learn offers two other types of regression, Ridge and Lasso, whichhelp with reducing the magnitude of the coefficientsand reduces overfitting. Using regularization,determine if your model improves using the Ridge or Lasso regression. See which alphavalues provide the best results. Describe your results

In [15]:
def make_ridge(data,normalize=False,alpha=1.0):
    
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = Ridge(normalize=normalize,alpha=alpha).fit(data.values[:,4:], life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg,score
ridge,score = make_ridge(train)
eval_model(ridge,train)

,Axis Name,Coefficient
0,Country,-1.670757e-02
1,Year,9.077189e-02
2,Status,-2.232568e-02
3,Life expectancy,4.391881e-04
4,Adult Mortality,-3.267372e-03
5,infant deaths,-5.947556e-06
6,Alcohol,2.964199e-02
7,percentage expenditure,-6.905520e-02
8,Hepatitis B,8.671247e-03
9,Measles,6.208256e-02


,Country,Standard Deviation
0,Greece,0.726127
1,Morocco,2.637561
2,Jamaica,0.887599
3,Seychelles,2.950191
4,France,2.173949
...,...,...
118,Costa Rica,0.000000
119,Syrian Arab Republic,0.000000
120,Bosnia and Herzegovina,5.827143
121,Albania,0.000000


In [16]:
def make_lasso(data,normalize=False,alpha=1.0):
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = Lasso(normalize=normalize,alpha=alpha).fit(data.values[:,4:],
        life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg,score
lasso,score = make_lasso(train)
eval_model(lasso,train)

,Axis Name,Coefficient
0,Country,-2.071057e-02
1,Year,7.346134e-02
2,Status,9.354387e-03
3,Life expectancy,3.031067e-04
4,Adult Mortality,-0.000000e+00
5,infant deaths,1.164543e-06
6,Alcohol,4.871699e-02
7,percentage expenditure,-5.713800e-02
8,Hepatitis B,1.075218e-02
9,Measles,0.000000e+00


,Country,Standard Deviation
0,Greece,1.137059
1,Morocco,2.781800
2,Jamaica,1.512939
3,Seychelles,0.924716
4,France,2.064289
...,...,...
118,Costa Rica,0.000000
119,Syrian Arab Republic,0.000000
120,Bosnia and Herzegovina,1.705170
121,Albania,0.000000


## Tuning alpha

In [17]:
max_lasso = 0
max_lasso_a=0
best_lasso=""
max_ridge = 0
max_ridge_a=0
best_ridge=""
for a_int in range(1,100,1):
    a= (float (a_int))/10.0
    lasso_model,lasso_score=make_lasso(train,alpha=a)
    if(lasso_score>max_lasso):
        max_lasso=lasso_score
        max_lasso_a=a
        best_lasso=lasso_model
    ridge_model,ridge_score=make_ridge(train,alpha=a)
    if(ridge_score>max_ridge):
        max_ridge=ridge_score
        max_ridge_a=a
        best_ridge=ridge_model
display(Markdown("### Max Lasso Alpha: "+str(max_lasso_a)))
display(Markdown("### Max Ridge Alpha: "+str(max_ridge_a)))

### Max Lasso Alpha: 0.1

### Max Ridge Alpha: 0.1

## e. Cross validate your model(you can use Scikit’s crossvalidation feature). Describe how yourcrossvalidated performance compares with best model. What does the cross validation results tell you about your model.

In [18]:
def eval_cross_validate(model,train,model_name:str):
    # dropping rows from test dataset
    test_cleaned = drop_rows(train)
    x = test_cleaned.values[:,4:]
    y=test_cleaned.values[:,3]
    #print(x)
    #print(y)
    scores = cross_val_score(model,x,y,cv=5)
    display(Markdown("## Mean "+model_name+ " Score: "+format(scores.mean(),".2f")))
eval_cross_validate(best_lasso,train,"Lasso")
eval_cross_validate(best_ridge,train,"Ridge")

## Mean Lasso Score: 0.81

## Mean Ridge Score: 0.82

The cross validated scores are very simmilar to the scores generated when using a score on the entire dataset. This means that my test dataset is likely representitive of the population dataset. 